In [67]:
import pandas as pd
import numpy as np
execfile('../src/moldev_utils.py')

In [68]:
py = pd.read_csv('../output/condition_summary.csv')

In [69]:
def tablify(df,cols):
    """ Return DataFrame with n + 2 columns, where n is length of cols.
        The first n columns are those specified in cols. 
        The next column contains the remaining column names of the input DataFrame.
        The last column containt the values from these remaining columns. """
    return df.set_index(cols).stack().reset_index(level = range(len(cols) + 1)).rename(columns = {"level_2":"Parameter",0:"Value"})

In [70]:
py2 = tablify(py,['Condition','Function'])

In [71]:
# py2['Function'] = py2['Function'].map(lambda x: x.replace("mean","avg"))
py2 = py2[py2['Function'].isin(['avg','std','count','sem'])]

In [72]:
ex = pd.read_csv('./summary/excel_summary.csv')\
    .dropna(how='all',axis=1)\
    .rename(columns = {'Unnamed: 0': 'Condition', 'Unnamed: 1': 'Function'})\
    .rename(columns = rename_col)
ex['Condition'] = ex['Condition'].fillna(method='ffill')
ex['Function'] = ex['Function'].map(lambda x: x.split(' ')[-1].lower())

In [73]:
ex2 = tablify(ex,['Condition','Function'])
ex2 = ex2[ex2['Function'].isin(['avg','std','count','sem'])]

In [74]:
comparison = pd.merge(py2,ex2,on=['Condition','Function','Parameter'])

In [75]:
comparison['reldiff'] = comparison.apply(lambda x: np.abs((x.Value_x - x.Value_y) / float(x.Value_y)) * 100, axis = 1)

In [76]:
comparison.sort('reldiff', ascending = False).head()

,Condition,Function,Parameter,Value_x,Value_y,reldiff
880,143B 11.1 uM drug A,std,Area_Coloc_Avg,0.019244,0.019252,0.043892
910,143B 11.1 uM drug A,sem,Area_Coloc_Avg,0.003513,0.003515,0.043891
1270,143B 5.55 uM drug A,sem,Area_Coloc_Avg,0.001871,0.001871,0.012133
1240,143B 5.55 uM drug A,std,Area_Coloc_Avg,0.024470,0.024473,0.012131
850,143B 11.1 uM drug A,avg,Area_Coloc_Avg,0.089468,0.089459,0.009427


In [77]:
assert comparison.reldiff.max() < 0.1, "Relative error between Excel and Python summaries is greater than 0.1% for some values."